# Weather Scraper

This notebook demonstrates how to scrape weather data from http://weather.com. We will look at the 10-day forecast\*, scrape the high temperatures, and then generate a graph to visualize them.


\*But clearly, there are more than 10 days in the "10-day forecast".

### First we read in the data and create the *soup*.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# it is good practice to identify yourself
# change the user-agent as appropriate
headers = {"user-agent": "WeatherBot/1.0"}
page = requests.get("https://weather.com/weather/5day/l/USCT0260:1:US", headers = headers)

if page.status_code != requests.codes.ok :
    print("Request was not successful, status code:", page.status_code)
    exit()
    
# Parse page using BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

### Scrape the data

In order to scrape the high temperature information we must understand the structure of the web page. The following observations are key:
- Each 'row' containing the forecast information is stored in a *details* tag; the 10-day forecast starts with the second *details* element. The *details* tag is described here: https://www.w3schools.com/tags/tag_details.asp
- Within each 'row', the date is inside of an *h2* tag.
- Within each 'row', the temperature is in a *span* tag.

Let's first look at the first day of the 10 day forecast, which will have an id of 'detailIndex1':

In [ ]:
day1 = soup.find('details', id = 'detailIndex1')
day1

Within this element, the date is in the *h2* tag:

In [ ]:
day1.h2.text

Within this element, the high temperature is in the first *span* tag:

In [ ]:
day1.span.text

Now extract the data, creating a list containing the days and a list containing the high temps. For the high temps, we remove the degree symbol and convert to an integer so that it can be graphed. We also ensure that the temp is a digit, to prevent any errors in the conversion.

In [ ]:
details = soup.find_all('details')
days = []
temps = []

# for each detail (starting with the 2nd one)
for d in details[1:] :
    
    # extract the date and the high temp
    date = d.h2.text
    temp = d.span.text
    
    # print the information
    print(date, ': ', temp, sep = '')

    # format temp and add to the list
    temp = temp.replace('°', '')
    if temp.isdigit() :
        days.append(date)
        temps.append(int(temp))

We will visualize the data by creating a bar plot with days on the $x$-axis and high temperatures on the $y$-axis. In order to do this, we first create a *pandas* data frame (table). The data frame is created by specifying a dictionary, where the keys will be the column names and the values will be the values for the corresponding column.

In [ ]:
df= pd.DataFrame(data = {"days":days, "hi_temp":temps})
df

Once we have a data frame, we can use the *plot.bar* method to create a bar plot from the data, where we specify the $x$ and $y$ values of the plot. 

In [ ]:
plt = df.plot.bar(x = "days", y = "hi_temp", 
            title = "Willimantic High Temps, 10 day forecast",
            legend = False)

plt.set_xlabel("\nDay of the Week (Date)")
plt.set_ylabel("Temperature (Degrees Fahrenheit)")
None